In [36]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import keras
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.callbacks import Callback
from keras.models import Model
from Data import DataLoader
from DataLoader import Data
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from sklearn.feature_selection import mutual_info_classif
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
df=Data.load_data('music.csv')

path_filename=r'C:\Users\PROMIT\Desktop\Music_Genre\music_01.csv'
label_dict={
    'final_blues':0,
    'final_classical':1,
    'final_country':2,
    'final_disco':3,
    'final_hiphop':4,
    'final_jazz':5,
    'final_metal':6,
    'final_pop':7,
    'final_reggae':8,
    'final_rock':9
}
df=Data.load_data('music.csv')

trainx,testx,trainy,testy=Data.prepare_data(df,label_dict={
    'final_blues':0,
    'final_classical':1,
    'final_country':2,
    'final_disco':3,
    'final_hiphop':4,
    'final_jazz':5,
    'final_metal':6,
    'final_pop':7,
    'final_reggae':8,
    'final_rock':9
})    
sc=StandardScaler()

trainx=sc.fit_transform(trainx)
testx=sc.transform(testx)
num_epochs=40

class Model:
    def __init__(self,input_shape,num_classes):
        self.input_shape=input_shape
        self.num_classes=num_classes

    def build_model(self):
       inputs=tf.keras.Input(shape=self.input_shape)
       layer1=tf.keras.layers.Dense(512,activation='relu')(inputs)
       layer2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
       layer3=tf.keras.layers.Dense(2048,activation='relu')(layer2)
       layer4=tf.keras.layers.Dropout(rate=0.4)(layer3)
       layer5=tf.keras.layers.Dense(512,activation='relu')(layer4)
       layer6=tf.keras.layers.Dropout(rate=0.5)(layer5)
       layer5=tf.keras.layers.Dense(self.num_classes,activation='softmax')(layer6)
       model=tf.keras.Model(inputs=inputs,outputs=layer5)

       return model
    
        

model=Model(input_shape=(341),num_classes=10)
model=model.build_model()
print(model.summary())

params={
    'batch_size':64,
    'epochs':40,
    'validation_split':0.2
}

def train_model(model,trainx,trainy,params):
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')    
    model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics='acc')
    model.fit(trainx,trainy,batch_size=params['batch_size'],validation_split=params['validation_split'],epochs=params['epochs'],callbacks=[earlyStopping, mcp_save, reduce_lr_loss])
    

train_model(model,trainx,trainy,params)

# preds=model(testx)
# preds=np.array(preds)
# preds=np.argmax(preds,axis=1)
# preds=np.array(preds,dtype='int64')

Data Loaded, Shape : 10000 ,343
Data Loaded, Shape : 10000 ,343
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 341)]             0         
                                                                 
 dense_65 (Dense)            (None, 512)               175104    
                                                                 
 dense_66 (Dense)            (None, 1024)              525312    
                                                                 
 dense_67 (Dense)            (None, 2048)              2099200   
                                                                 
 dropout_26 (Dropout)        (None, 2048)              0         
                                                                 
 dense_68 (Dense)            (None, 512)               1049088   
                                                            

In [39]:
preds=model.predict(testx)
f1_score(testy,np.argmax(preds,axis=1),average='macro')

63/63 [==============================] - 0s 7ms/step


0.944004132825814

In [16]:
from MMR_Feature_Extract import MMR_Selection

In [17]:
data=MMR_Selection(trainx,trainy,alpha=0.5)
indices,trainx=data.get_cols(matrix=trainx,num_features=100)
trainx=trainx[:,indices]
def apply_valid(valid_data,index_list):
    valid_data=valid_data[:,index_list]
    return valid_data
validx=apply_valid(valid_data=testx,index_list=indices)

In [18]:
def apply_valid(valid_data,index_list):
    valid_data=valid_data[:,index_list]
    return valid_data
validx=apply_valid(valid_data=testx,index_list=indices)

In [19]:
class Model:
    def __init__(self,input_shape,num_classes):
        self.input_shape=input_shape
        self.num_classes=num_classes

    def build_model(self):
       inputs=tf.keras.Input(shape=self.input_shape)
       layer1=tf.keras.layers.Dense(512,activation='relu')(inputs)
       layer2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
       layer3=tf.keras.layers.Dense(2048,activation='relu')(layer2)
       layer4=tf.keras.layers.Dropout(rate=0.4)(layer3)
       layer5=tf.keras.layers.Dense(512,activation='relu')(layer4)
       layer6=tf.keras.layers.Dropout(rate=0.5)(layer5)
       layer5=tf.keras.layers.Dense(self.num_classes,activation='softmax')(layer6)
       model=tf.keras.Model(inputs=inputs,outputs=layer5)

       return model
def train_model(model,trainx,trainy,params):
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.compressed_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')    
    model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics='acc')
    model.fit(trainx,trainy,batch_size=params['batch_size'],validation_split=params['validation_split'],epochs=params['epochs'],callbacks=[earlyStopping, mcp_save, reduce_lr_loss])
    
       
        
model=Model(input_shape=(100),num_classes=10)
model=model.build_model()
print(model.summary())
train_model(model,trainx,trainy,params)

preds=model(validx)
preds=np.array(preds)
preds=np.argmax(preds,axis=1)
preds=np.array(preds,dtype='int64')

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_50 (Dense)            (None, 512)               51712     
                                                                 
 dense_51 (Dense)            (None, 1024)              525312    
                                                                 
 dense_52 (Dense)            (None, 2048)              2099200   
                                                                 
 dropout_20 (Dropout)        (None, 2048)              0         
                                                                 
 dense_53 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_21 (Dropout)        (None, 512)               0  

In [ ]:
preds=np.argmax(preds,axis=1)
f1_score(testy,preds,average='macro')

In [29]:
preds=np.argmax(preds,axis=1)
f1_score(testy,preds,average='macro')


0.9386503753594369

In [33]:
from sklearn.decomposition import PCA
pc=PCA(n_components=100)
trainx=pc.fit_transform(trainx)
testx=pc.transform(testx)

class Model:
    def __init__(self,input_shape,num_classes):
        self.input_shape=input_shape
        self.num_classes=num_classes

    def build_model(self):
       inputs=tf.keras.Input(shape=self.input_shape)
       layer1=tf.keras.layers.Dense(512,activation='relu')(inputs)
       layer2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
       layer3=tf.keras.layers.Dense(2048,activation='relu')(layer2)
       layer4=tf.keras.layers.Dropout(rate=0.4)(layer3)
       layer5=tf.keras.layers.Dense(512,activation='relu')(layer4)
       layer6=tf.keras.layers.Dropout(rate=0.5)(layer5)
       layer5=tf.keras.layers.Dense(self.num_classes,activation='softmax')(layer6)
       model=tf.keras.Model(inputs=inputs,outputs=layer5)

       return model
def train_model(model,trainx,trainy,params):
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
    mcp_save = ModelCheckpoint('.compressed_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')    
    model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics='acc')
    model.fit(trainx,trainy,batch_size=params['batch_size'],validation_split=params['validation_split'],epochs=params['epochs'],callbacks=[earlyStopping, mcp_save, reduce_lr_loss])
    
       
        
model=Model(input_shape=(100),num_classes=10)
model=model.build_model()
print(model.summary())
train_model(model,trainx,trainy,params)

preds=model(validx)
preds=np.array(preds)
preds=np.argmax(preds,axis=1)
preds=np.array(preds,dtype='int64')

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_60 (Dense)            (None, 512)               51712     
                                                                 
 dense_61 (Dense)            (None, 1024)              525312    
                                                                 
 dense_62 (Dense)            (None, 2048)              2099200   
                                                                 
 dropout_24 (Dropout)        (None, 2048)              0         
                                                                 
 dense_63 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_25 (Dropout)        (None, 512)               0  

In [34]:
from sklearn.metrics import f1_score
preds=model.predict(testx)


63/63 [==============================] - 1s 10ms/step


In [35]:
preds=np.argmax(preds,axis=1)
f1_score(testy,preds,average='macro')

0.9296928823575117